<a href="https://colab.research.google.com/github/JKrusche1/PhARIS/blob/main/PhARIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://raw.githubusercontent.com/JKrusche1/PhARIS/main/phage_logo.png" height="200" align="right" style="height:200px">

##**PhARIS - v1.0**
##**Ph**age **A**ureus **R**BP **I**dentification **S**ystem

Tool for rapid detection of _Staphylococcus aureus_ phage receptor-binding proteins (RBPs) by blast homology search of a query with known _S. aureus_ RBP clusters.

Created by Janes Krusche

2023 - manuscript in preparation


In [ ]:
#@title Install dependencies
#@markdown #### Install biopython, blast+ and RBP reference files

from google.colab import files
import io
import os
import json

print("Installing biopython...")
!pip install biopython > /dev/null
print("Done")
print("Installing blast...")
!apt-get install ncbi-blast+ > /dev/null
print("Done")
print("Downloading reference files...")
if not os.path.exists("files"):
  !git clone https://github.com/JKrusche1/PhARIS.git &>/dev/null
  !mv PhARIS/files /content/
  !rm -r PhARIS
  if not os.path.exists("files/blast_main"):
      !mkdir files/blast_main
  if not os.path.exists("files/blast_rev"):
      !mkdir files/blast_rev
  # Read the master dictionary from the JSON file
  with open("files/RBP.json", "r") as json_file:
      master_dict = json.load(json_file)

  # Access individual dictionaries using their names
  clusterlist_main = master_dict["clusterlist_main"]
  clusterlist_rev = master_dict["clusterlist_rev"]
  FACS_values = master_dict["FACS_values"]
  FACS_SEM = master_dict["FACS_SEM"]
  RBP_defi = master_dict["RBP_defi"]
print("Done")

In [ ]:
#@title ### Search for S. aureus phage RBP
#@markdown Once executed, a button will appear that lets you upload your phage genome in fasta format.\
#@markdown After the upload is finished, the script will search for an _S. aureus_ binding RBP in your genome.\
#@markdown If an RBP is found, you will get the closest matching RBP of the clustering, as well as the identity to this RBP.\
#@markdown To find out where your RBP is located in the phylogenetic tree, go to the links provided and search for the closest match.\
#@markdown [Primary RBP tree](https://itol.embl.de/tree/4652552332411677924980)\
#@markdown [Secondary RBP tree](https://itol.embl.de/tree/46525523312761688382866)

#@markdown To search another genome, simply rerun this cell and upload another file.

#Import libraries
from Bio import SeqIO
import os
import glob
import pathlib
import pandas as pd
import sys
import time
import matplotlib.pyplot as plt
import numpy as np

#Upload file
# Prompt the user to upload a file
uploaded = files.upload()

# Check if any files were uploaded
if uploaded:
    # If files were uploaded, execute the following block of code
    print('File uploaded successfully.')

    # Save the uploaded file to the "files" directory with the name "genome.fasta"
    with open(os.path.join('files', 'genome.fasta'), 'wb') as f:
        # Write the contents of the uploaded file to the newly created "genome.fasta" file
        # The "next(iter(uploaded))" extracts the name of the uploaded file
        # The "uploaded[next(iter(uploaded))]" fetches the contents of the uploaded file
        f.write(uploaded[next(iter(uploaded))])
else:
    # If no files were uploaded, execute the following block of code
    print('No file uploaded.')

    # Exit the program
    sys.exit()



#Detect ORFs in query
print("Searching for RBP...")
for seq_record in SeqIO.parse("files/genome.fasta", "fasta"):
    orfseq = []
    orflen = []
    orfnr = []
    x = 0
    file = seq_record.seq
    for strand, nuc in [(+1, file), (-1, file.reverse_complement())]:
        for frame in range(3):
            length = 3 * ((len(file)-frame) // 3) #Multiple of three
            for pro in nuc[frame:frame+length].translate(11).split("*"):
                  splitlocal = pro.find('M')
                  seq_final = pro[splitlocal:]
                  if len(seq_final) >= 100:
                    x = x+1
                    orflen.append(len(seq_final))
                    seq_final = str(seq_final)
                    seq_final = '\n'.join(seq_final[i:i+60] for i in range(0, len(seq_final), 60))
                    orfseq.append(str(seq_final))
                    orfnr.append(x)
if len(orfseq) == 0:
    print("No ORFs were detected.")
    sys.exit()
else:
    with open('files/query_db.fasta', 'w') as f:
        for i in range(len(orfseq)):
            f.write(">" + seq_record.id + "_orf" + str(orfnr[i]) + " - length " + str(orflen[i]) + "\n")
            f.write(orfseq[i] + "\n")

#Create database and blast with known RBPs
!makeblastdb -in files/query_db.fasta -dbtype prot -out files/protein_db > /dev/null;
for file in glob.glob("files/RBPs_main/*.fasta"):
        filename = file.replace("files/RBPs_main/","")
        filename = filename.replace(".fasta","")
        !blastp -query {file} -db files/protein_db -outfmt 6 -out files/blast_main/{filename}_blast.txt
for file in glob.glob("files/RBPs_rev/*.fasta"):
        filename = file.replace("files/RBPs_rev/","")
        filename = filename.replace(".fasta","")
        !blastp -query {file} -db files/protein_db -outfmt 6 -out files/blast_rev/{filename}_blast.txt

#Extract primary RBP from blast
result = pd.DataFrame()
for filename in glob.glob("files/blast_main/*_blast.txt"):
    df = pd.read_csv(filename, sep='\t', header = None)
    df.columns = ['query acc.ver', 'subject', 'identity', 'alignment length', 'mismatches', 'gap opens', 'q. start', 'q. end', 's. start', 's. end', 'evalue', 'bit score']
    df["lengthper"] = df["alignment length"]/df['alignment length'].iloc[0]
    df = df[df.lengthper >=0.8]
    df = df[df.identity > 80]
    df.loc[df['lengthper'] > 1, 'lengthper'] = 1
    df = df.reset_index(drop=True)

    if df.empty:
      x = x
    else:
      result = pd.concat([result, df.sort_values('identity', ascending=False).head(1)], ignore_index=True)
      finaldf = df

# Graph settings
x_labels = ['SA ΔtagO', 'SA WT', 'SA ΔtarM', 'SA ΔtarS', 'SA ΔtarM/S', 'SE 1457']
colors = ['#F4FFC0', '#F4FFC0', '#F4FFC0', '#F4FFC0', '#F4FFC0', '#90BFF9']

#Visualize primary RBP
print("-----------------------------------------------------------------------------------------------------------")
RBPs_found = 0
if result.empty:
    print("No primary S. aureus binding phage RBP could be detected.")
else:
    RBPs_found = 1
    print("\nPrimary S. aureus binding phage RBP detected\n")
    for i in range(len(result)):
      phageRBP = result.iloc[i]["query acc.ver"]
      for name, lst in clusterlist_main.items():
        for item in lst:
            if phageRBP in item:
                cluster = name
      if "phiK " in cluster:
        if "subgroup a" in cluster:
          graphname = "phiK"
        elif "subgroup b" in cluster:
          graphname = "phiStab20"
        elif "subgroup c" in cluster:
          graphname = "phiSA012"
        elif "subgroup d" in cluster:
          graphname = "phiPG2021-10"
        elif "subgroup e" in cluster:
          graphname = "phiPG2021-17"
        elif "subgroup f" in cluster:
          graphname = "not tested"
        elif "outlier" in cluster:
          graphname = "not tested"
      else:
        graphname = cluster
      if graphname in RBP_defi:
        RBP_match = RBP_defi[graphname][0]
        match_row = finaldf.loc[finaldf['query acc.ver'] == RBP_match]
        identity_value = match_row.iloc[0]['identity']
      print("-\033[1mPRIMARY RBP " + str(i+1) + ": " + result.iloc[i]["subject"] + "\033[0m")
      print("-Clustered in group: \033[1m" + cluster + "\033[0m")
      print("-Highest sequence identity with RBP of phage \033[4m\033[1m" + result.iloc[i]["query acc.ver"].split("_orf")[0] + "\033[0m: " + str(result.iloc[i]["identity"]) + "%")
      print("-Overlap with main RBP of phage " + result.iloc[i]["query acc.ver"].split("_orf")[0] + ": " + str(result.iloc[i]["lengthper"]*100) + "%")
      print("-ORFs can be found in 'files/query_db.fasta'.\n")
      with open('files/query_db.fasta', 'r') as f:
          query_db = f.read()
          query_db = query_db.split(result.iloc[0]["subject"])[-1]
          query_db = query_db.split(">")[0]
          print(">" + result.iloc[0]["subject"] + query_db + "\n")
      y_values = [0,0,0,0,0,0]
      if graphname in FACS_values:
          print("\033[1mIdentity of RBP " + result.iloc[i]["subject"] + " with RBP of " + graphname + ": " + str(np.round(identity_value,2)) + "%\033[0m")
          y_values = FACS_values[graphname]
          sem = FACS_SEM[graphname]
          ind = np.arange(6)
          # Create the bar chart with the desired size
          fig, ax = plt.subplots(figsize=(3.5, 3.5))

          # Create the bar plot
          ax.bar(x_labels, y_values, align='center', alpha=1, ecolor="k", color=colors, edgecolor="k",
                 linewidth=1.5)
          plotline1, caplines1, barlinecols1 = ax.errorbar(ind, y_values, yerr=sem, lolims=True,
                                                           capsize = 0, ls='None', color="k", alpha=0.5,
                                                           linewidth=1)

          caplines1[0].set_marker('_')
          caplines1[0].set_markersize(10)
          # Set the axis labels and title
          ax.set_ylim(bottom=0)
          ax.set_xlabel('')
          ax.set_ylabel('Median fluorescence intensity [AU]')
          ax.set_title('Binding of ' + graphname + " main RBP")
          ax.set_xticks(range(len(x_labels)))
          ax.set_xticklabels(x_labels, rotation=45, ha='right')
      else:
          print(f"No binding pattern available for {graphname}")
      plt.show()
      print(f"\n-For more information, find the closest matching RBP under {'https://itol.embl.de/tree/4652552332411677924980'}")
if 'df' in locals():
    del df
if 'result' in locals():
    del result

#Extract secondary RBP from blast
result = pd.DataFrame()
for filename in glob.glob("files/blast_rev/*_blast.txt"):
    df = pd.read_csv(filename, sep='\t', header = None)
    df.columns = ['query acc.ver', 'subject', 'identity', 'alignment length', 'mismatches', 'gap opens', 'q. start', 'q. end', 's. start', 's. end', 'evalue', 'bit score']
    df["lengthper"] = df["alignment length"]/df['alignment length'].iloc[0]
    df = df[df.lengthper >=0.8]
    df = df[df.identity > 80]
    df.loc[df['lengthper'] > 1, 'lengthper'] = 1
    df = df.reset_index(drop=True)
    if df.empty:
      x = x
    else:
      result = pd.concat([result, df.sort_values('identity', ascending=False).head(1)], ignore_index=True)
      finaldf = df

#Visualize secondary RBP
print("-----------------------------------------------------------------------------------------------------------")
if result.empty:
    print("No secondary S. aureus binding phage RBP could be detected.")
    if RBPs_found == 0:
      print("-----------------------------------------------------------------------------------------------------------")
      print("\033[1mNo RBPs found. This phage might not be able to infect S. aureus or is an outliers that does not cluster.\033[0m")
else:
    print("\nSecondary S. aureus binding phage RBP detected\n")
    for i in range(len(result)):
      phageRBP = result.iloc[i]["query acc.ver"]
      for name, lst in clusterlist_rev.items():
        for item in lst:
            if phageRBP in item:
                cluster = name
      if "phiK_reversible " in cluster:
        if "subgroup a" in cluster:
          graphname = "phiK_reversible"
        elif "subgroup b" in cluster:
          graphname = "phiPG2021-10_reversible"
        elif "outlier" in cluster:
          graphname = "not tested"
      else:
        graphname = cluster
      if graphname in RBP_defi:
        RBP_match = RBP_defi[graphname][0]
        match_row = finaldf.loc[finaldf['query acc.ver'] == RBP_match]
        identity_value = match_row.iloc[0]['identity']
      print("-\033[1mREVERSIBLE RBP " + str(i+1) + ": " + result.iloc[i]["subject"] + "\033[0m")
      print("-Clustered in group: \033[1m" + cluster + "\033[0m")
      print("-Highest sequence identity with RBP of phage \033[4m\033[1m" + result.iloc[i]["query acc.ver"]
            .split("_orf")[0] + "\033[0m: " + str(result.iloc[i]["identity"]) + "%")
      print("-Overlap with main RBP of phage " + result.iloc[i]["query acc.ver"].split("_orf")[0] + ": " + str(result.iloc[i]["lengthper"]*100) + "%")
      print("-ORFs can be found in 'files/query_db.fasta'.\n")

      with open('files/query_db.fasta', 'r') as f:
          query_db = f.read()
          query_db = query_db.split(result.iloc[0]["subject"])[-1]
          query_db = query_db.split(">")[0]
          print(">" + result.iloc[0]["subject"] + query_db + "\n")
      y_values = [0,0,0,0,0,0]
      if graphname in FACS_values:
          print("\033[1mIdentity of RBP " + result.iloc[i]["subject"] + " with RBP of " + graphname + ":" + str(np.round(identity_value,2)) + "%\033[0m")
          y_values = FACS_values[graphname]
          sem = FACS_SEM[graphname]
          ind = np.arange(6)
          # Create the bar chart with the desired size
          fig, ax = plt.subplots(figsize=(3.5, 3.5))

          # Create the bar plot
          ax.bar(x_labels, y_values, align='center', alpha=1, ecolor="k", color=colors, edgecolor="k",
                 linewidth=1.5)
          plotline1, caplines1, barlinecols1 = ax.errorbar(ind, y_values, yerr=sem, lolims=True,
                                                           capsize = 0, ls='None', color="k", alpha=0.5,
                                                           linewidth=1)

          caplines1[0].set_marker('_')
          caplines1[0].set_markersize(10)
          # Set the axis labels and title
          ax.set_ylim(bottom=0)
          ax.set_xlabel('')
          ax.set_ylabel('Median fluorescence intensity [AU]')
          ax.set_title('Binding of ' + graphname + " RBP")
          ax.set_xticks(range(len(x_labels)))
          ax.set_xticklabels(x_labels, rotation=45, ha='right')
      else:
          print(f"No binding pattern available for {graphname}")
      plt.show()
      print(f"\n-For more information, find the closest matching RBP under {'https://itol.embl.de/tree/46525523312761688382866'}")